# Laboratorio 4 - Clínica de los Alpes PipeLine

Integrantes del Equipo:

* Santiago Bobadilla - 201820728
* Juan José Beltrán Ruiz - 201819446

Referencias Usadas:

https://towardsdatascience.com/scikit-learn-pipeline-tutorial-with-parameter-tuning-and-cross-validation-e5b8280c01fb

https://towardsdatascience.com/data-science-quick-tip-003-using-scikit-learn-pipelines-66f652f26954

https://towardsdatascience.com/data-science-quick-tip-004-using-custom-transformers-in-scikit-learn-pipelines-89c28c72f22a

# Librerias Usadas

*   *pandas*: Lectura de datos originales
*   *sklearn.linear_model*: Creación de la regresión lineal
*   *joblib*: Cargar y guardar el modelo
*   *sklearn.metrics*: Rendimiento del modelo.

In [ ]:
# Lectura de datos.
import pandas as pd

# Regresion lineal
from sklearn.linear_model import LinearRegression

# Importar/ Exportar modelos
from joblib import dump, load

# Para determinar el rendimiento del modelo con la métrica R2
from sklearn.metrics import r2_score

# Datos

Los JSON que se van a pasar por medio del API vienen inicialmente de los datos originales en CSV los cuales se muestran a continuación. Así como se explico en el laboratorio 3, la única correción necesaria para datos inicialmente es remover valores nulos o ilogicos. 

Dado lo obtenido en el análisis y limpieza, se puede concluir que los datos iniciales son adecuados.

In [ ]:
# Se cargan los datos. 
df = pd.read_csv('202210_Laboratorio3_data_DatosTrain.csv', sep=',', encoding = 'utf-8')

# Eliminación del indentificador, ya que no aporta información alguna para el proceso de regresión
df = df.drop(['Unnamed: 0'], axis=1)

# Revisamos los primeros registros de la base de datos para revisar ausencias y demás
df.head()

,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 10-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,65.0,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,59.9,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,59.9,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,59.5,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,59.2,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


# Modelo + PipeLine

En primera instancia, se requiere importar dos elementos más de la libreria *sklearn.pipeline* los cuales son:

*   *PipeLine*: Con el cual podremos realizar el Pipe Line completo funcional.
*   *FeatureUnion*: "This estimator applies a list of transformer objects in parallel to the input data, then concatenates the results. This is useful to combine several feature extraction mechanisms into a single transformer."

Con esto en mente, y dado que lo único necesario para realizar un correcto funcionamiento de nuestra regresión lineal era eliminar columnas con base a un criterio, decidimos implementar una transformación propia: **columnDropperTransformer**.

Dicha clase se encarga de eliminar variables por medio de una lista. No obstante, puede ser programada para deducir las variables con base a mejora de significancia de las metricas, o ruptura de alguno de los supuestos. Por el momento, dado que realizamos una análisis en la laboratorio pasado de los supuestos y variables no significativas el PipeLine ya eliminara las variables no necesarias. 

Así, uniendo las librerias y la clase propia se crea el PipeLine que se va a usar.

In [ ]:
# Librerias
from sklearn.pipeline import Pipeline, FeatureUnion

In [ ]:
# Eliminar Columnas
class columnDropperTransformer():
    def __init__(self,columns):
        self.columns=columns

    def transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

    def fit(self, X, y=None):
        return self

In [ ]:
# Uso del pipeline para exportar el modelo
pipe = Pipeline([
    ("features", FeatureUnion([
        ("columnDropper", columnDropperTransformer(['infant deaths',
                                                    'percentage expenditure',
                                                    'Hepatitis B',
                                                    'under-five deaths',
                                                    'Population',
                                                    'thinness 5-9 years']))
    ])),
    ('classifier', LinearRegression())
])

In [ ]:
# Se selecciona la variable objetivo
Y = df['Life expectancy']

# Del conjunto de datos se elimina la variable objetivo
X = df.drop(['Life expectancy'], axis=1)

In [ ]:
# Ajustar el PipeLine.
#     --> Esto SIEMPRE se debe hacer antes de realizar un PREDICT
pipe.fit(X,Y)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('columnDropper',
                                                 <__main__.columnDropperTransformer object at 0x7fdaee0f7b50>)])),
                ('classifier', LinearRegression())])

In [ ]:
# Se obtienen las predicciones del modelo sobre el conjunto de entrenamiento.
y = pipe.predict(X)
# Se obtienen las métricas a partir de la predicción y la base de evaluación (valores reales).
print('R²: %.2f' % r2_score(Y, y))

R²: 0.66


# Regresion_PipeLine

* Santiago Bobadilla - 201820728
* Juan José Beltrán Ruiz - 201819446

---

## Carpetas

1. *requierments*: Contiene un archivo **.txt** donde se puede encontrar los paquetes de Python necesarios (normalmente no instalados por defecto) que son necesarios para el proyecto. Igualmente dicha información se da en un archivo **.bat** con el fin de poder realizar una rapida ejecusión en *Windows*.

2. *data*: Contiene un archivo **.json** con la información utilizada para hacer *fit* al modelo. Dicha información es la misma que se pasa en *Postman*.

3. *postman*: Contiene dos archivos **.json**. Uno con las pruebas de *train* y *predict* en localhost y otro desplegado en Heroku. 

4. *assests*: Si bien puede estar vacio, contiene el modelo PipeLine guardado como **.joblib**.

## Archivos

Todos los archivos explican su funcionamiento internamente por medio de su respectiva documentación:

*   *DataModel.py*: Incluye los modelos usados para el *train* y *predict*.
*   *PredictionModel.py*: Incluye la implementación del **PipeLine** y sus debidos metodos.
*   *main.py*: Incliye la ejecusión del **API** que permite realizar las pruebas necesarias por medio de la aplicación de **POSTMAN**: https://www.postman.com/ 

## Ejecutar

1. Descargar el repositorio e instalar lo necesario con base en la carpeta: **requierments**.

2. Correr el Main por medio del siguiente comando en **terminal**:

```
uvicorn main:app --reload
```

3. Revisar que en consola tenga dos posibles respuestas en texto plano:

```
PipeLine Creado        or        PipeLine Cargado
```

4. Correr las pruebas en **POSTMAN**.

## POSTMAN

Para correr las pruebas de manera local se debe usar cualquiera de los siguientes links dependiendo de la necesidad:

*   *Train*:http://127.0.0.1:8000/train
*   *Predict*: http://127.0.0.1:8000/predict

Cada uno de dichos links requiere un archivo **json** pasado como **raw** en **body** por medio de Postamn bajo la siguiente estructura: 

***TRAIN***

Es de suma importancia notar que el JSON de la carpeta data es un listado de registros. Para que el API funcione, dicha lista debe estar guardada en la variable del JSON 'data'. Aparte es necesaria la información de la variable Y, como sus explicativas X.

```
{
  "data": [
    {
      "life_expentancy": 0,
      "adult_mortality": 0,
      "infant_deaths": 0,
      "alcohol": 0,
      "percentage_expenditure": 0,
      "hepatitis_B": 0,
      "measles": 0,
      "bmi": 0,
      "under_five_deaths": 0,
      "polio": 0,
      "total_expenditure": 0,
      "diphtheria": 0,
      "hiv_aids": 0,
      "gdp": 0,
      "population": 0,
      "thinness_10_19_years": 0,
      "thinness_5_9_years": 0,
      "income_composition_of_resources": 0,
      "schooling": 0
    },
    {
      "life_expentancy": 0,
      "adult_mortality": 0,
      "infant_deaths": 0,
      "alcohol": 0,
      "percentage_expenditure": 0,
      "hepatitis_B": 0,
      "measles": 0,
      "bmi": 0,
      "under_five_deaths": 0,
      "polio": 0,
      "total_expenditure": 0,
      "diphtheria": 0,
      "hiv_aids": 0,
      "gdp": 0,
      "population": 0,
      "thinness_10_19_years": 0,
      "thinness_5_9_years": 0,
      "income_composition_of_resources": 0,
      "schooling": 0
    }
  ]
}
```

***PREDICT***

Para predecir se pasan las variables explicativas X y nada más. Se debe aclarar que si el modelo acaba de ser creado con el fin de lograr predecir debe realizar primero un ajusto, por tanto, si bien el **API** verifica dicha afirmación, se sugiere correr primero las pruebas de *train* y luego las de *predict* como aparecen en las colecciones de Postman suministardas. 

```
{
  "adult_mortality": 0,
  "infant_deaths": 0,
  "alcohol": 0,
  "percentage_expenditure": 0,
  "hepatitis_B": 0,
  "measles": 0,
  "bmi": 0,
  "under_five_deaths": 0,
  "polio": 0,
  "total_expenditure": 0,
  "diphtheria": 0,
  "hiv_aids": 0,
  "gdp": 0,
  "population": 0,
  "thinness_10_19_years": 0,
  "thinness_5_9_years": 0,
  "income_composition_of_resources": 0,
  "schooling": 0
}
```

## Escenarios 

Los resultados de los escenarios se encuentran en el **.pdf** en la carpeta **data**, así como se suministran las colecciones ejecutables donde se puede revisar lo dicho. En total se tienen 5 pruebas de predict, y 1 de train.

0. *Train*: Carga una tabla de datos y devuelve la métrica de R^2.
1. *Predict*: Prueba y valor lógico correctos.
2. *Predict*: Prueba y valor lógico correctos.
3. *Predict*: Prueba correcta y valor lógico incorrecto.
4. *Predict*: Prueba correcta y valor lógico incorrecto.
5. *Predict*: Prueba incorrecta.

Como conclusiones de porque un resultado es incorrecto se tiene un explicación visible de los datos. Cuando una variable esta fuera de su rango normal (definase normal, como dentro de los limites estipulados por el negocio y la naturaleza para dicho componente) la predicción es errorea. Un segundo resultado no evidente por lo obtenido en el análisis del laboratio 3 puede ser entrenar el modelo con datos que tengan problemas dentro de los supuestos. 

Como conclusión cuando la aplicación falla se debe unicamente a la violación de los parametros del **json** pasados en postamn, es decir, falta algun parametro. 

Como medida de solución, se propone: *Realizar una validación por medio del API de los datos, buscando comprobar si todas las variables estan dentro del rango normal, o en su defecto, usar un SimpleImputer de la libreria Sklearn. Así se logra validar que la información presentada sea correcta. Segundo, programar filtros personalizados que permitan remover columnas o transformar los datos cuando un supuesto se viole con el fin de saber que los resultados del modelo son fiables. Tercero, y ya respecto al API, permiter que no se pasen ciertos campos sin marcar error, pero que dichos campos sean luego imputados por medio de alguna tecnica.*